In [ ]:
import pandas as pd
import os

# === File Config ===
daily_file = "MLB_Combined_Odds_Results_2025-05-07.csv"
abbrev_file = "MLB_Teams_Template.xlsx"
master_file = "master_template.parquet"

# === Load Game-Level File and Team Abbreviations ===
df = pd.read_csv(daily_file)
abbrev_df = pd.read_excel(abbrev_file).rename(columns={"City and Team": "team_name", "Abbreviation": "team_abbr"})
abbrev_map = dict(zip(abbrev_df["team_name"], abbrev_df["team_abbr"]))

# === Transform to Team-Level Rows ===
team_rows = []

for _, row in df.iterrows():
    if pd.isna(row["home_score"]) or pd.isna(row["away_score"]):
        continue

    total_score = row["home_score"] + row["away_score"]
    hit_over = total_score > row["total_line"] if pd.notna(row["total_line"]) else None

    for team_type in ["home", "away"]:
        is_home = team_type == "home"
        team = row[f"{team_type}_team"]
        opponent = row[f"{'away' if is_home else 'home'}_team"]
        team_score = row[f"{team_type}_score"]
        opp_score = row[f"{'away' if is_home else 'home'}_score"]
        moneyline = row[f"moneyline_{team_type}"]

        row_data = {
            "game_id": row["game_id"],
            "game_date_et": pd.to_datetime(row["game_date"]),
            "start_time_et": pd.to_datetime(row["start_time_et"]),
            "team": team,
            "team_abbr": abbrev_map.get(team),
            "opponent": opponent,
            "opponent_abbr": abbrev_map.get(opponent),
            "is_home": is_home,
            "home_score": row["home_score"],
            "away_score": row["away_score"],
            "run_diff": team_score - opp_score,
            "won_game": team_score > opp_score,
            "hit_over": hit_over,
            "team_streak": None,
            "Wins": None,
            "Losses": None,
            "Win_Pct": None,
            "Win_Streak": None,
            "Loss_Streak": None,
            "merge_key": f"{team}_{row['game_date']}",
            "team_odds": moneyline,
            "opponent_odds": row[f"moneyline_{'away' if is_home else 'home'}"],
            "is_home_odds": is_home,
            "Run_Line": None,
            "Spread_Price": None,
            "Opp_Spread_Price": None,
            "Total": row["total_line"],
            "Over_Price": row["over_odds"],
            "Under_Price": row["under_odds"],
            "h2h_own": None,
            "h2h_opp": None,
            "team_abbr_odds": abbrev_map.get(team),
            "opponent_abbr_odds": abbrev_map.get(opponent)
        }

        # Add innings (preserve full 1–9 view)
        for i in range(1, 10):
            row_data[f"home_{i}"] = row.get(f"home_{i}")
            row_data[f"away_{i}"] = row.get(f"away_{i}")

        team_rows.append(row_data)

# === Create DataFrame
team_df = pd.DataFrame(team_rows)

# === Append to Master Parquet ===
if os.path.exists(master_file):
    master_df = pd.read_parquet(master_file)
    master_df["game_date_et"] = pd.to_datetime(master_df["game_date_et"])
    master_df["start_time_et"] = pd.to_datetime(master_df["start_time_et"])
    combined_df = pd.concat([master_df, team_df], ignore_index=True)
else:
    combined_df = team_df

# === Ensure consistent data types
combined_df["team_odds"] = pd.to_numeric(combined_df["team_odds"], errors="coerce")
combined_df["opponent_odds"] = pd.to_numeric(combined_df["opponent_odds"], errors="coerce")
combined_df["Over_Price"] = pd.to_numeric(combined_df["Over_Price"], errors="coerce")
combined_df["Under_Price"] = pd.to_numeric(combined_df["Under_Price"], errors="coerce")
combined_df["Total"] = pd.to_numeric(combined_df["Total"], errors="coerce")
combined_df["run_diff"] = pd.to_numeric(combined_df["run_diff"], errors="coerce")


combined_df.to_parquet(master_file, index=False)
print(f"✅ Appended {len(team_df)} team-level rows to: {master_file}")


✅ Appended 30 team-level rows to: master_template.parquet


<ipython-input-12-261eb1546d29>:83: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([master_df, team_df], ignore_index=True)
